##### Copyright 2019 Google LLC.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Mesh Segmentation using Feature Steered Graph Convolutions
<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/graphics/blob/master/tensorflow_graphics/notebooks/mesh_segmentation_demo.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/graphics/blob/master/tensorflow_graphics/notebooks/mesh_segmentation_demo.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

In [ ]:
!pip install tensorflow_graphics

Now that TensorFlow Graphics and dependencies are installed, let's import everything needed to run the demos contained in this notebook.

In [ ]:
import glob
import os
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

# from tensorflow_graphics.nn.layer import graph_convolution as graph_conv
from tensorflow_graphics.notebooks import mesh_segmentation_dataio as dataio
# from tensorflow_graphics.notebooks import mesh_viewer

Instructions for updating:
non-resource variables are not supported in the long term


Note this notebook works best in Graph mode.

### Fetch model files and data

For convenience, we provide a pre-trained model. Let's now download a pre-trained model checkpoint and the test data. The meshes are generated using Unity Multipurpose Avatar system [UMA](https://assetstore.unity.com/packages/3d/characters/uma-2-unity-multipurpose-avatar-35611).

In [ ]:
path_to_model_zip = tf.keras.utils.get_file(
    'model.zip',
    origin='https://storage.googleapis.com/tensorflow-graphics/notebooks/mesh_segmentation/model.zip',
    extract=True)

path_to_data_zip = tf.keras.utils.get_file(
    'data.zip',
    origin='https://storage.googleapis.com/tensorflow-graphics/notebooks/mesh_segmentation/data.zip',
    extract=True)

local_model_dir = os.path.join(os.path.dirname(path_to_model_zip), 'model')
test_data_files = [
    os.path.join(
        os.path.dirname(path_to_data_zip),
        'data/Dancer_test_sequence.tfrecords')
]

## Load and visualize test data

For graph convolutions, we need a *weighted adjacency matrix* denoting the mesh
connectivity. Feature-steered graph convolutions expect self-edges in the mesh
connectivity for each vertex, i.e. the diagonal of the weighted adjacency matrix
should be non-zero. This matrix is defined as:
```
A[i, j] = w[i,j] if vertex i and vertex j share an edge,
A[i, i] = w[i,i] for each vertex i,
A[i, j] = 0 otherwise.
where, w[i, j] = 1/(degree(vertex i)), and sum(j)(w[i,j]) = 1
```
Here degree(vertex i) is the number of edges incident on a vertex (including the
self-edge). This weighted adjacency matrix is stored as a SparseTensor.

We will load the test meshes from the test [tf.data.TFRecordDataset](https://www.tensorflow.org/api_docs/python/tf/data/TFRecordDataset)
downloaded above. Each mesh is stored as a
[tf.Example](https://www.tensorflow.org/api_docs/python/tf/train/Example), with
the following fields:

*   'num_vertices': Number of vertices in each mesh.
*   'num_triangles': Number of triangles in each mesh.
*   'vertices': A [V, 3] float tensor of vertex positions.
*   'triangles': A [T, 3] integer tensor of vertex indices for each triangle.
*   'labels': A [V] integer tensor with segmentation class label for each
    vertex.

where 'V' is number of vertices and 'T' is number of triangles in the mesh. As
each mesh may have a varying number of vertices and faces (and the corresponding
connectivity matrix), we pad the data tensors with '0's in each batch.

For details on the dataset pipeline implementation, take a look at
mesh_segmentation_dataio.py.

Let's try to load a batch from the test TFRecordDataset, and visualize the first
mesh with each vertex colored by the part label.

In [ ]:
test_io_params = {
    'is_training': False,
    'sloppy': False,
    'shuffle': True,
}
test_tfrecords = test_data_files


input_graph = tf.Graph()
with input_graph.as_default():
  mesh_load_op = dataio.create_input_from_dataset(
      dataio.create_dataset_from_tfrecords, test_tfrecords, test_io_params)
  with tf.Session() as sess:
    test_mesh_data, test_labels = sess.run(mesh_load_op)

# input_mesh_data = {
#     'vertices': test_mesh_data['vertices'][0, ...],
#     'faces': test_mesh_data['triangles'][0, ...],
#     'vertex_colors': mesh_viewer.SEGMENTATION_COLORMAP[test_labels[0, ...]],
# }
# print("num of vertices slices: ",len(test_mesh_data['vertices'][0]),"read from file: ",test_mesh_data['num_vertices'])
# print(input_mesh_data)
# print(type(input_mesh_data))
# print()
# input_viewer = mesh_viewer.Viewer(input_mesh_data)

Instructions for updating:
Use `tf.data.Dataset.interleave(map_func, cycle_length, block_length, num_parallel_calls=tf.data.AUTOTUNE)` instead. If sloppy execution is desired, use `tf.data.Options.experimental_deterministic`.


In [ ]:
print(test_mesh_data['num_vertices'])

[2652 2659 2654 2663 2660 2658 2665 2662]


In [ ]:
for i in range(len(test_mesh_data['num_vertices'])):
  myvertex = test_mesh_data['vertices'][i]
  myface = test_mesh_data['triangles'][i]
  num_vertex = test_mesh_data['num_vertices'][i]
  num_face = test_mesh_data['num_triangles'][i]
  outfile = 'OFF\n'+ str(num_vertex) +' '+ str(num_face)+' '+ str(num_vertex+num_face-2) + '\n'
  for j in range(num_vertex):
    outfile += str(myvertex[j][0]) + ' '+str(myvertex[j][1]) + ' '+str(myvertex[j][2]) + '\n'
  for j in range(num_face):
    outfile += '3 '+str(myface[j][0])+' '+str(myface[j][1])+' '+str(myface[j][2])+'\n'
  myfile = open('/content/drive/MyDrive/mesh_seg/data/Dancer_test_sequence'+str(i)+'.off','w')
  myfile.write(outfile)
  myfile.close()
# vertex0 = test_mesh_data['vertices'][0]
# face0 = test_mesh_data['triangles'][0]
# num_vertex0 = test_mesh_data['num_vertices'][0]
# num_faces0 = test_mesh_data['num_triangles'][0]
# print(len(vertex0),len(face0))
# print(test_mesh_data['num_vertices'][0],test_mesh_data['num_triangles'][0])
# outfile = 'OFF\n'+ str(num_vertex0) +' '+ str(num_faces0)+' '+ str(num_vertex0+num_faces0-2) + '\n'
# for i in range(num_vertex0):
#   outfile += str(vertex0[i][0]) + ' '+str(vertex0[i][1]) + ' '+str(vertex0[i][2]) + '\n'
# # print(outfile)
# # outfile = ''
# for i in range(num_faces0):
#   outfile += '3 '+ str(face0[i][0]) + ' '+str(face0[i][1]) + ' '+str(face0[i][2]) + '\n'
# print(outfile)

In [ ]:
# myfile = open('/content/drive/MyDrive/mesh_seg/data/mymodel0.off','w')
# myfile.write(outfile)
# myfile.close()